In [ ]:
import pandas as pd
from wordcloud import WordCloud
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk import word_tokenize          
from nltk.stem.porter import PorterStemmer
import nltk
from nltk.tokenize import RegexpTokenizer


# 1. Wordclouds
## Loading the data

In [ ]:
emails = pd.read_csv("hillary-clinton_emails/Emails.csv")
emails.head()

First we inspect what data we are given. For the wordcloud it would be wise to use only the "ExtractedBodyText" column.

In [ ]:
emails.columns.values

## Removing emails without body text

In [ ]:
emails.size #Size of the dataset before

In [ ]:
emails = emails[pd.notnull(emails['ExtractedBodyText'])]
emails.size #Size of the dataset after

## Generating the wordcloud

Converting the ExtractedBodyText into strings we can use for the wordcloud.

In [ ]:
text = emails.ExtractedBodyText.to_string();

In [ ]:
wordcloud = WordCloud().generate(text)
plt.imshow(wordcloud)
plt.axis("off")
plt.show()


## Improving the wordcloud

To improve the wordcloud we will go through the process of removing stopwords, although it seems that the original Wordcloud already removes these. Then we will make sure to have only alphabetical characters and use stemming to convert words into their original base form.

In [ ]:
stemmer = PorterStemmer()
sw = stopwords.words("english")

In [ ]:
def tokenize(text):
    tokens = nltk.word_tokenize(text) # Tokenizing the text
    tokens = [i for i in tokens if i.isalpha()] #Keeping only strings with alphabetic characters
    tokens = [i for i in tokens if i.lower() not in sw] #Removing stopwords
    stems = stem_tokens(tokens, stemmer) #Stemming the words
    return stems  #returns a list

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed 

In [ ]:
improvedText = str(tokenize(text)).replace("'", "") #Convert from list to string object without single quotes

In [ ]:
wc = WordCloud().generate(improvedText)
plt.imshow(wc)
plt.axis("off")
plt.show()

When comparing the two wordclouds, we see that they are largely similar. In the first cloud we have 'nFriday' and 'nSunday' as backslashes are converted to 'n'.  In the second cloud some words are shortened to their stems such as 'Novemb' and 'relea'.

# 2. Sentiment analysis

# 3. Topic modeling

# Bonus - Communication graph